Copyright The Numerical Algorithms Group Limited 1994.
 ntube.input
 Generalized tubes.
 The functions in this file draw a 2-d curve in the normal
 planes around a 3-d curve.  The computations are all done
 numerically in machine-precision floating point for efficiency.


In [ ]:
R3 := Point DoubleFloat                  -- Points in 3-Space

In [ ]:
R2 := Point DoubleFloat                  -- Points in 2-Space

In [ ]:
S := Segment Float              -- Draw ranges

In [ ]:
ThreeCurve := DoubleFloat -> R3          -- type of a space curve function

In [ ]:
TwoCurve := (DoubleFloat, DoubleFloat) -> R2      -- type of a plane curve function

 Frenet frames define a coordinate system around a point on a space curve


In [ ]:
Surface := (DoubleFloat, DoubleFloat) -> R3   -- type of a parameterized surface function

 Holds current Frenet frame for a point on a curve


In [ ]:
FrenetFrame := Record(value: R3, tagent: R3, normal: R3, binormal: R3)

 compile, don't interpret functions


In [ ]:
frame: FrenetFrame

 Draw a generalized tube.
 ntubeDraw(spaceCurve, planeCurve, u0..u1, t0..t1)
 draws planeCurve int the normal planes of spaceCurve.  u0..u1 specifies
 the paramter range of the planeCurve and t0..t1 specifies the parameter
 range of the spaceCurve.  Additionally the plane curve function takes
 as a second parameter the current parameter of the spaceCurve.  This
 allows the plane curve to evolve as it goes around the space curve.
 see "page5.input" for an example of this.


In [ ]:
)set fun compile on

In [ ]:
ntubeDraw: (ThreeCurve, TwoCurve, S, S) -> VIEW3D

 ntuberDrawOpt is the same as ntuberDraw, but takes optional
 parameters which it passes to the draw command.


In [ ]:
ntubeDraw(spaceCurve, planeCurve, uRange, tRange) ==
  ntubeDrawOpt(spaceCurve, planeCurve, uRange, tRange, []$List DROPT)

In [ ]:
ntubeDrawOpt: (ThreeCurve, TwoCurve, S, S, List DROPT) -> VIEW3D

 nfrenetFrame(c, t, delT) numerically computes the Frenet Frame
 about the curve c at t.  delT is a small number used to
 compute derivatives.


In [ ]:
ntubeDrawOpt(spaceCurve, planeCurve, uRange,  tRange, l) ==
  delT:DoubleFloat := (hi(tRange) - lo(tRange))/10000
  oldT:DoubleFloat := lo(tRange) - 1
  fun := ngeneralTube(spaceCurve, planeCurve, delT, oldT)
  draw(fun, uRange, tRange, l)

 nGeneralTube(spaceCurve, planeCurve, delT, oltT)
 creates a function which can be passed to the system draw command.
 The function is a parameterized surface for the general tube
 around the spaceCurve.  delT is a small number used to compute
 derivatives, and oldT is used to hold the current value of the
 t parameter for the spaceCurve.  This is an efficiency measure
 to ensure that frames are only computed once for every value of t.


In [ ]:
nfrenetFrame(c, t, delT) ==
  f0 := c(t)
  f1 := c(t+delT)
  t0 := f1 - f0                -- the tangent
  n0 := f1 + f0
  b := cross(t0, n0)           -- the binormal
  n := cross(b,t0)             -- the normal
  ln := length n
  lb := length b
  ln = 0 or lb = 0 => error "Frenet Frame not well defined"
  n := (1/ln)*n                -- make into unit length vectors
  b := (1/lb)*b
  [f0, t0, n, b]$FrenetFrame

In [ ]:
ngeneralTube: (ThreeCurve, TwoCurve, DoubleFloat, DoubleFloat) -> Surface

In [ ]:
ngeneralTube(spaceCurve, planeCurve, delT, oldT) ==
  free frame
  (v:DoubleFloat, t: DoubleFloat): R3 +->
    if (t ~= oldT) then
      frame := nfrenetFrame(spaceCurve, t, delT)
      oldT := t
    p := planeCurve(v, t)
    frame.value + p.1*frame.normal + p.2*frame.binormal